<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Michael", "transactions": [{"transaction-id": 1, "amount": 366}, {"transaction-id": 2, "amount": 381}, {"transaction-id": 55, "amount": 346}, {"transaction-id": 90, "amount": 344}, {"transaction-id": 102, "amount": 359}, {"transaction-id": 155, "amount": 380}, {"transaction-id": 174, "amount": 391}, {"transaction-id": 277, "amount": 393}, {"transaction-id": 329, "amount": 382}, {"transaction-id": 505, "amount": 374}, {"transaction-id": 739, "amount": 322}, {"transaction-id": 817, "amount": 408}, {"transaction-id": 927, "amount": 401}, {"transaction-id": 1013, "amount": 379}, {"transaction-id": 1066, "amount": 373}, {"transaction-id": 1072, "amount": 405}, {"transaction-id": 1100, "amount": 370}, {"transaction-id": 1209, "amount": 362}, {"transaction-id": 1327, "amount": 351}, {"transaction-id": 1377, "amount": 366}, {"transaction-id": 1510, "amount": 378}, {"transaction-id": 1530, "amount": 372}, {"transaction-id": 1545, "amount": 352}, {"transaction-id": 1811, "am

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Michael", "transactions": [{"transaction-id": 1, "amount": 366}, {"transaction-id": 2, "amount": 381}, {"transaction-id": 55, "amount": 346}, {"transaction-id": 90, "amount": 344}, {"transaction-id": 102, "amount": 359}, {"transaction-id": 155, "amount": 380}, {"transaction-id": 174, "amount": 391}, {"transaction-id": 277, "amount": 393}, {"transaction-id": 329, "amount": 382}, {"transaction-id": 505, "amount": 374}, {"transaction-id": 739, "amount": 322}, {"transaction-id": 817, "amount": 408}, {"transaction-id": 927, "amount": 401}, {"transaction-id": 1013, "amount": 379}, {"transaction-id": 1066, "amount": 373}, {"transaction-id": 1072, "amount": 405}, {"transaction-id": 1100, "amount": 370}, {"transaction-id": 1209, "amount": 362}, {"transaction-id": 1327, "amount": 351}, {"transaction-id": 1377, "amount": 366}, {"transaction-id": 1510, "amount": 378}, {"transaction-id": 1530, "amount": 372}, {"transaction-id": 1545, "amount": 352}, {"transaction-id": 1811, "am

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Michael',
  'transactions': [{'transaction-id': 1, 'amount': 366},
   {'transaction-id': 2, 'amount': 381},
   {'transaction-id': 55, 'amount': 346},
   {'transaction-id': 90, 'amount': 344},
   {'transaction-id': 102, 'amount': 359},
   {'transaction-id': 155, 'amount': 380},
   {'transaction-id': 174, 'amount': 391},
   {'transaction-id': 277, 'amount': 393},
   {'transaction-id': 329, 'amount': 382},
   {'transaction-id': 505, 'amount': 374},
   {'transaction-id': 739, 'amount': 322},
   {'transaction-id': 817, 'amount': 408},
   {'transaction-id': 927, 'amount': 401},
   {'transaction-id': 1013, 'amount': 379},
   {'transaction-id': 1066, 'amount': 373},
   {'transaction-id': 1072, 'amount': 405},
   {'transaction-id': 1100, 'amount': 370},
   {'transaction-id': 1209, 'amount': 362},
   {'transaction-id': 1327, 'amount': 351},
   {'transaction-id': 1377, 'amount': 366},
   {'transaction-id': 1510, 'amount': 378},
   {'transaction-id': 1530, 'amount': 372},
   

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 21,
  'name': 'Alice',
  'transactions': [{'transaction-id': 72, 'amount': 36},
   {'transaction-id': 282, 'amount': 48},
   {'transaction-id': 1226, 'amount': 58},
   {'transaction-id': 3487, 'amount': 61},
   {'transaction-id': 4641, 'amount': 66},
   {'transaction-id': 5615, 'amount': 45},
   {'transaction-id': 5942, 'amount': 61},
   {'transaction-id': 6267, 'amount': 52},
   {'transaction-id': 6750, 'amount': 68},
   {'transaction-id': 8041, 'amount': 49},
   {'transaction-id': 8940, 'amount': 68}]},
 {'id': 24,
  'name': 'Alice',
  'transactions': [{'transaction-id': 211, 'amount': 282},
   {'transaction-id': 375, 'amount': 302},
   {'transaction-id': 607, 'amount': 277},
   {'transaction-id': 903, 'amount': 298},
   {'transaction-id': 1205, 'amount': 282},
   {'transaction-id': 1302, 'amount': 263},
   {'transaction-id': 1316, 'amount': 282},
   {'transaction-id': 1683, 'amount': 288},
   {'transaction-id': 2003, 'amount': 279},
   {'transaction-id': 2467, 'amount': 263}

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 11},
 {'name': 'Alice', 'count': 30},
 {'name': 'Alice', 'count': 13},
 {'name': 'Alice', 'count': 64},
 {'name': 'Alice', 'count': 79})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(11, 30, 13, 64, 79)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

42.516

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 72, 'amount': 36},
  {'transaction-id': 282, 'amount': 48},
  {'transaction-id': 1226, 'amount': 58},
  {'transaction-id': 3487, 'amount': 61},
  {'transaction-id': 4641, 'amount': 66},
  {'transaction-id': 5615, 'amount': 45},
  {'transaction-id': 5942, 'amount': 61},
  {'transaction-id': 6267, 'amount': 52},
  {'transaction-id': 6750, 'amount': 68},
  {'transaction-id': 8041, 'amount': 49},
  {'transaction-id': 8940, 'amount': 68}],
 [{'transaction-id': 211, 'amount': 282},
  {'transaction-id': 375, 'amount': 302},
  {'transaction-id': 607, 'amount': 277},
  {'transaction-id': 903, 'amount': 298},
  {'transaction-id': 1205, 'amount': 282},
  {'transaction-id': 1302, 'amount': 263},
  {'transaction-id': 1316, 'amount': 282},
  {'transaction-id': 1683, 'amount': 288},
  {'transaction-id': 2003, 'amount': 279},
  {'transaction-id': 2467, 'amount': 263},
  {'transaction-id': 2986, 'amount': 269},
  {'transaction-id': 3105, 'amount': 280},
  {'transaction-id': 3945, '

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 72, 'amount': 36},
 {'transaction-id': 282, 'amount': 48},
 {'transaction-id': 1226, 'amount': 58})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(36, 48, 58)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

832.5890958697902

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 110), ('Alice', 120), ('Alice', 130), ('Alice', 140), ('Bob', 109), ('Bob', 119), ('Bob', 129), ('Bob', 137), ('Charlie', 66), ('Charlie', 72), ('Charlie', 78), ('Charlie', 84), ('Dan', 154), ('Dan', 168), ('Dan', 182), ('Dan', 196), ('Edith', 88), ('Edith', 96), ('Edith', 104), ('Edith', 112), ('Frank', 110), ('Frank', 120), ('Frank', 130), ('Frank', 140), ('George', 97), ('George', 103), ('George', 114), ('George', 123), ('Hannah', 44), ('Hannah', 48), ('Hannah', 52), ('Hannah', 56), ('Ingrid', 110), ('Ingrid', 120), ('Ingrid', 130), ('Ingrid', 140), ('Jerry', 143), ('Jerry', 156), ('Jerry', 168), ('Jerry', 182), ('Kevin', 98), ('Kevin', 105), ('Kevin', 109), ('Kevin', 118), ('Laura', 82), ('Laura', 88), ('Laura', 96), ('Laura', 101), ('Michael', 110), ('Michael', 120), ('Michael', 130), ('Michael', 140), ('Norbert', 153), ('Norbert', 168), ('Norbert', 316), ('Oliver', 121), ('Oliver', 132), ('Oliver', 143), ('Oliver', 153), ('Patricia', 116), ('Patricia', 125), ('Patricia

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 500), ('Bob', 494), ('Charlie', 300), ('Dan', 700), ('Edith', 400), ('Frank', 500), ('George', 437), ('Hannah', 200), ('Ingrid', 500), ('Jerry', 649), ('Kevin', 430), ('Laura', 367), ('Michael', 500), ('Norbert', 637), ('Oliver', 549), ('Patricia', 517), ('Quinn', 381), ('Ray', 250), ('Sarah', 478), ('Tim', 350), ('Ursula', 450), ('Victor', 580), ('Wendy', 448), ('Xavier', 600), ('Yvonne', 530), ('Zelda', 550)]
CPU times: user 172 ms, sys: 1.57 ms, total: 174 ms
Wall time: 620 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Michael,"[{'transaction-id': 1, 'amount': 366}, {'trans..."
1,1,Wendy,"[{'transaction-id': 48, 'amount': 118}, {'tran..."
2,2,Hannah,"[{'transaction-id': 321, 'amount': 3025}, {'tr..."
3,3,Tim,"[{'transaction-id': 421, 'amount': 1453}, {'tr..."
4,4,Jerry,"[{'transaction-id': 6, 'amount': 931}, {'trans..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 234 ms, sys: 20.5 ms, total: 255 ms
Wall time: 1.89 s


name
Alice      500
Bob        494
Charlie    300
Dan        700
Edith      400
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Michael', 'amount': 366, 'transaction-id': 1},
 {'id': 0, 'name': 'Michael', 'amount': 381, 'transaction-id': 2},
 {'id': 0, 'name': 'Michael', 'amount': 346, 'transaction-id': 55})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Michael,366,1
1,0,Michael,381,2
2,0,Michael,346,55
3,0,Michael,344,90
4,0,Michael,359,102


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 209 ms, sys: 8.46 ms, total: 218 ms
Wall time: 1.48 s


name
Alice       21258
Bob         17514
Charlie     17756
Dan         17623
Edith       13393
Frank       21895
George      20243
Hannah       3411
Ingrid      15925
Jerry       24062
Kevin       24465
Laura       19602
Michael     17079
Norbert     30471
Oliver      16215
Patricia    26444
Quinn       18460
Ray          8638
Sarah       13305
Tim         19465
Ursula      13439
Victor      18974
Wendy       18297
Xavier      35375
Yvonne      16255
Zelda       30436
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()